<a href="https://colab.research.google.com/github/patricksabry1/42028-Deep-Learning/blob/master/character_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Character Recognition model Using OpenCV & deep CNN

In [6]:
# import the pre-trained CNN and its related preprocessing method
from tensorflow.keras.applications.xception import Xception, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Dense, Conv2D, Flatten, GlobalAveragePooling2D, MaxPooling2D, SpatialDropout2D
from tensorflow.keras.models import Model
import os
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
cd /content/gdrive/My Drive/42028-DL-CNN-2020/assignment-3/

/content/gdrive/My Drive/42028-DL-CNN-2020/assignment-3


# Pre-process cropped images

In [22]:
from google.colab.patches import cv2_imshow
import os

# Apply canny edge detection 
def auto_canny(image, sigma=0.33):
    v = np.median(image)
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edged_image = cv.Canny(image, lower, upper)
 
    return edged_image

# Crops characters out of numerplate 
def crop_ctrs(img):
    ret, mask = cv2.threshold(grayimage, 254, 255, cv2.THRESH_BINARY)

    cv2.imshow('mask', mask)
    cv2.waitKey(0)

    image, contours, hierarchy = cv2.findContours(mask, cv2.RETR_LIST, 
    cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:

        if cv2.contourArea(contour) < 200:
            continue

        rect = cv2.minAreaRect(contour)
        box = cv2.boxPoints(rect)

        ext_left = tuple(contour[contour[:, :, 0].argmin()][0])
        ext_right = tuple(contour[contour[:, :, 0].argmax()][0])
        ext_top = tuple(contour[contour[:, :, 1].argmin()][0])
        ext_bot = tuple(contour[contour[:, :, 1].argmax()][0])

        roi_corners = np.array([box], dtype=np.int32)

        cv2.polylines(bounding_box_image, roi_corners, 1, (255, 0, 0), 3)
        cv2.imshow('image', bounding_box_image)
        cv2.waitKey(0)

        cropped_image = grayimage[ext_top[1]:ext_bot[1], ext_left[0]:ext_right[0]]
        cv2.imwrite('crop.jpg', cropped_image)

path = "/content/gdrive/My Drive/42028-DL-CNN-2020/assignment-3/cropped_dataset/plates/train/"
bounding_boxes = []
counter = 0
for image_path in os.listdir(path):
    if counter < 200:
        full_image_path = os.path.join(path, image_path)
        img = cv.imread(full_image_path)

        # Loop through each image, apply pre-processing & localize onto each character
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        thresh_inv = cv.adaptiveThreshold(gray,255,cv.ADAPTIVE_THRESH_MEAN_C,cv.THRESH_BINARY_INV,39,1)
        edges = auto_canny(thresh_inv)
        ctrs, _ = cv.findContours(edges.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
        sorted_ctrs = sorted(ctrs, key=lambda ctr: cv.boundingRect(ctr)[0])

        img_area = img.shape[0]*img.shape[1]

        # Get bounding box co-ordinates for image cropping
        for i, ctr in enumerate(sorted_ctrs):
            x, y, w, h = cv.boundingRect(ctr)
            roi_area = w*h
            roi_ratio = roi_area/img_area

            if((roi_ratio >= 0.08) and (roi_ratio < 0.16)):
                    if ((h>1.2*w) and (3*w>=h)):
                        cv.rectangle(img,(x,y),( x + w, y + h ),(90,0,255), 1)
                        bounding_boxes.append((x,y,w,h))
                        counter += 1

print(bounding_boxes)
# Crop bounding boxes and save into new dir
count = 0
for box in bounding_boxes:
        x,y,w,h = box
        ROI = img[y:y+h, x:x+w]
        cv.imwrite('/content/gdrive/My Drive/42028-DL-CNN-2020/assignment-3/cropped_dataset/characters/validation/char_{}.png'.format(str(count)), ROI)
        count += 1
        cv2_imshow(ROI)
        cv.waitKey()

KeyboardInterrupt: ignored

# Generate images using PIL with similar font


In [2]:
from PIL import Image, ImageDraw, ImageFont

alphanumerics = ["A", "a","B","b","C","c","D","d","E","e","F","f","G","g","H","h","I","J","K","k","L","l","M","m","N","n","O","o","P","p","Q","q","R",
                "r","S","s","T","t","U","u","V","v","W","w","X","x","Y","y","Z","z","0","1","2","3","4","5","6","7","8","9","10"]

labels_map_dict = {}
def generate_images():
    counter = 0
    index = 0
    while counter < 1:
        for char in alphanumerics:
            labels_map_dict[index] = char
            index += 1
            img = Image.new('RGB', (512, 512), color = (255, 255, 255))
            fnt = ImageFont.truetype('/content/gdrive/My Drive/42028-DL-CNN-2020/assignment-3/cropped_dataset/characters/fonts/DIN 1451 Std Engschrift.otf', 34)
            d = ImageDraw.Draw(img)
            d.text((20,20), char, font=fnt, fill=(0, 0, 0))

            img.save('/content/gdrive/My Drive/42028-DL-CNN-2020/assignment-3/cropped_dataset/characters/test/char_{}_{}.png'.format(str(counter), char))
        counter += 1
generate_images()

print(labels_map_dict)

# initialise image generator on synthesized alphanumeric images, apply image augmentation - skew, rotation, scale

{0: 'A', 1: 'a', 2: 'B', 3: 'b', 4: 'C', 5: 'c', 6: 'D', 7: 'd', 8: 'E', 9: 'e', 10: 'F', 11: 'f', 12: 'G', 13: 'g', 14: 'H', 15: 'h', 16: 'I', 17: 'J', 18: 'K', 19: 'k', 20: 'L', 21: 'l', 22: 'M', 23: 'm', 24: 'N', 25: 'n', 26: 'O', 27: 'o', 28: 'P', 29: 'p', 30: 'Q', 31: 'q', 32: 'R', 33: 'r', 34: 'S', 35: 's', 36: 'T', 37: 't', 38: 'U', 39: 'u', 40: 'V', 41: 'v', 42: 'W', 43: 'w', 44: 'X', 45: 'x', 46: 'Y', 47: 'y', 48: 'Z', 49: 'z', 50: '0', 51: '1', 52: '2', 53: '3', 54: '4', 55: '5', 56: '6', 57: '7', 58: '8', 59: '9', 60: '10'}


In [3]:
def load_dataset(directory, target_size):
    images = []
    labels = []

    # Generate train images array & labels
    for image_path in os.listdir(directory):
        # Extract image data to numpy array
        full_image_path = os.path.join(directory, image_path)
        img = image.load_img(full_image_path, target_size=target_size)
        img_arr = image.img_to_array(img)
        img_arr = preprocess_input(img_arr) # Xception preprocessing
        images.append(img_arr)

        label = image_path.split('_')[2].replace(".png", "")
        for i, char in labels_map_dict.items():
            if char == label:
                labels.append(i)

    images = np.array(images)
    labels = np.array(labels)
    
    return images, labels

In [ ]:
import tensorflow as tf
import pandas as pd
import os
# Generate labels, split into train/test/val
train_path = "/content/gdrive/My Drive/42028-DL-CNN-2020/assignment-3/cropped_dataset/characters/train/"
test_path = "/content/gdrive/My Drive/42028-DL-CNN-2020/assignment-3/cropped_dataset/characters/test/"

target_size = (512,512)

X_train, y_train = load_dataset(train_path, target_size)
X_test, y_test = load_dataset(test_path, target_size)

# Onehotencode target classes
y_train = tf.keras.utils.to_categorical(
    y_train, num_classes=None, dtype='float32'
)

y_test = tf.keras.utils.to_categorical(
    y_test, num_classes=None, dtype='float32'
)

print(X_train.shape)
print(y_train.shape)

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rotation_range=10,  
        zoom_range = 0.10,  
        width_shift_range=0.1, 
        height_shift_range=0.1)

datagen.fit(X_train)
train_generator = datagen.flow(X_train, y_train, batch_size=32, seed=7)

(1769, 512, 512, 3)
(1769, 61)


# Deep CNN for character image classification

The localized numberplate characters are cropped and used as input data for an image classification CNN model which will have 35 target classes (all alphanumericals)

In [18]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import tensorflow as tf
from tensorflow import keras

# import the pre-trained CNN and its related preprocessing method
from tensorflow.keras.applications.xception import Xception, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Dense, Conv2D, Flatten, GlobalAveragePooling2D, MaxPooling2D, SpatialDropout2D
from tensorflow.keras.models import Model

input_shape=(512,512,3)
def make_model():
    # create new model
    base_model = Xception(include_top=False, weights="imagenet", input_shape=input_shape)

    # start from block13_sepconv2_activation to begin with 32x32 dimensionality
    x = base_model.layers[-13].output

    # add dropout2d layer to improve regularisation
    x = SpatialDropout2D(0.3)(x)
    
    # add additional conv2d layers to reduce dimensionality down to 1x1
    x = Conv2D(1024, (3, 3), activation="relu", padding="same")(x)
    x = Conv2D(1024, (1, 1), activation="relu")(x)

    x = Conv2D(256, (1, 1), activation="relu", padding="same")(x)
    x = Conv2D(512, (3, 3), strides=(2, 2), activation="relu")(x)

    x = Conv2D(128, (1, 1), activation="relu", padding="same")(x)
    x = Conv2D(256, (3, 3), strides=(2, 2), activation="relu")(x)

    x = Conv2D(128, (1, 1), activation="relu", padding="same")(x)
    x = Conv2D(256, (3, 3), strides=(2, 2), activation="relu")(x)

    x = Conv2D(128, (1, 1), activation="relu", padding="same")(x)
    x = Conv2D(256, (3, 3), strides=(2, 2), activation="relu")(x)

    # FC layers and 35 neuron output layer for softmax character classification
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation="relu")(x)
    x = Dense(512, activation="relu")(x)

    x = Dense(y_train.shape[1], activation="softmax")(x)

    # this is the model to be trained
    model = Model(inputs=base_model.input, outputs=x)

    # freeze the base_model layers
    for layer in base_model.layers[:-13]:
        layer.trainable = False
    
    print(model.summary())
    
    # compile model
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['accuracy'])
    
    return model

model = make_model()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 255, 255, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 255, 255, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 255, 255, 32) 0           block1_conv1_bn[0][0]            
____________________________________________________________________________________________

In [21]:
# setup callbacks to use during training
chk_path = os.path.join("/content/gdrive/My Drive/42028-DL-CNN-2020/assignment-3/cropped_dataset/characters/model_checkpoints/", "model.{epoch:02d}-val_acc{val_accuracy:.4f}.h5")

checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=chk_path,
    monitor="val_accuracy",
    mode="max",
    save_weights_only=False,
    save_best_only=True,
    verbose=1)

stopping_callback = keras.callbacks.EarlyStopping(
    monitor="val_accuracy",
    mode="max",
    min_delta=0.01,
    patience=16,
    verbose=1,
    restore_best_weights=True)

my_callbacks = [
    checkpoint_callback,
    stopping_callback
]

In [20]:
eps = 10

# train model
history = model.fit(train_generator, epochs=eps, verbose=1,
                    validation_data=(X_test, y_test), callbacks=my_callbacks)


Epoch 1/10
20/20 [==============================] - ETA: 0s - loss: 4.1143 - accuracy: 0.0082

KeyError: ignored

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16, 8))

# plot loss
ax[0].plot(history.history["loss"][1:], label="Loss (training)")
ax[0].plot(history.history["val_loss"][1:], label="Loss (validation)")
ax[0].set_title("Loss metrics")
ax[0].set_ylabel("Loss")
ax[0].set_xlabel("Epochs")
ax[0].legend(loc="upper left")

# plot mean iou
ax[1].plot(history.history["accuracy"], label="Accuracy (training)")
ax[1].plot(history.history["val_accuracy"], label="Accuracy (validation)")
ax[1].set_title("Accuracy metrics")
ax[1].set_ylabel("Mean IoU")
ax[1].set_xlabel("Epochs")
ax[1].legend(loc="upper left")

plt.show()